In [150]:
import pandas as pd
import numpy as np

In [151]:
df = pd.read_csv('incident_event_log.csv')

In [152]:
# Paddington with  TRYING TO REPLACE ALL ? in data frame and look how many columns have this 
def grab_number(value):
    #print (value) 
    if (value == "?").any():
        ### this code is saying go through each and every column and for each row try to find if that row is equal to Caller 2403
        return True
    return False

# df['caller_id'].str.split(" ").apply(lambda x: x[0]).value_counts()
#sum(df.apply(grab_number)) #.astype(str)
sum(df.apply(grab_number))

18

In [153]:
#change all ? marks to NAN values so we can manipulate them easier
df_nan = df.copy()
df_nan = df.replace('?',np.nan)

In [154]:
def grab_number(value):
    #print (value) 
    if (pd.isna(value)).any():
        ### isna is nan values
        return True
    return False

# df['caller_id'].str.split(" ").apply(lambda x: x[0]).value_counts()
#sum(df.apply(grab_number)) #.astype(str)
df_nan.apply(grab_number)

number                     False
incident_state             False
active                     False
reassignment_count         False
reopen_count               False
sys_mod_count              False
made_sla                   False
caller_id                   True
opened_by                   True
opened_at                  False
sys_created_by              True
sys_created_at              True
sys_updated_by             False
sys_updated_at             False
contact_type               False
location                    True
category                    True
subcategory                 True
u_symptom                   True
cmdb_ci                     True
impact                     False
urgency                    False
priority                   False
assignment_group            True
assigned_to                 True
knowledge                  False
u_priority_confirmation    False
notify                     False
problem_id                  True
rfc                         True
vendor    

In [155]:
#changing all objects to datetime values in the date calumns that we will use for our target columns
df_nan['opened_at']=pd.to_datetime(df_nan['opened_at'],dayfirst= True)
df_nan['resolved_at'] = pd.to_datetime(df_nan['resolved_at'], dayfirst = True)

In [156]:
# creating a difference column to show the amount of days it takes to resolve a ticket that will be used for the target column
df_nan['difference'] = (df_nan['resolved_at'] - df_nan['opened_at']).dt.days

In [157]:
#creates a target column of 0s and 1s based on the amount of days it took to resolve the incident.
#if the amount of days is greater than 2 or NAN it turns to 1 and if 2 or below it is a 0
df_nan['target_colum'] = df_nan.apply(lambda row: 1 if pd.notna(row['resolved_at'])
                                       and (row['resolved_at'] - row['opened_at']).days > 2 else 0, axis = 1)

In [158]:
# Creating a function to prioritize 'new' and 'active' incident states that will emliminate
"""Parmerter: takes grouped data frame and applys to each group of data
returns a data frame that only includes one instance of every value in the 'number' column
that has a correlating 'incident_state' value of 'New' if possible and then filters to 'active', 
while also including one instance of a 'number' that does not have new or active in 'incident_state' """
def new_active_incident_state(group):
    if (group['incident_state'] == 'New').any():
        return group[group['incident_state'] == 'New'].iloc[0]
    elif (group['incident_state'] == 'Active').any():
        return group[group['incident_state'] == 'Active'].iloc[0]
    else:
        return group.iloc[0]  # In case neither 'new' nor 'active' exists, retain the first occurrence

# Apply this function to the DataFrame by grouping by 'number'
df_new_active = df_nan.groupby('number',group_keys=False).apply(new_active_incident_state).reset_index(drop=True)
# View the result
#df_new_active

###warning note: group_keys = Flase is not yet apart of pandas and thus will not work, however by including group_keys = False exlcudes the 'number' column form being included in the apply results and should avoid altering the sturcture of the result


/var/folders/ys/j34f3pms18l5jc7tq2nl1qn00000gp/T/ipykernel_13857/1345615283.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_new_active = df_nan.groupby('number',group_keys=False).apply(new_active_incident_state).reset_index(drop=True)


In [159]:
#This shows that our filtered dataFrame retains all of the unique values in the 'number' column
duplicates_list = df_nan['number'].unique().tolist()
display(len(duplicates_list))
unique_list_check = df_new_active['number'].unique().tolist()
display(len(unique_list_check))

24918

24918

In [160]:
#this shows that we have a slightly imbalanced data set with 2x as many trues than falses, will probably have to use sampler
df_new_active['target_colum'].value_counts()

target_colum
0    16651
1     8267
Name: count, dtype: int64

In [161]:
#shows the total 8 possible values for filtering compared to the output of our filtered data frame
display(df_nan.value_counts('incident_state'))
display(df_new_active.value_counts('incident_state'))


incident_state
Active                38716
New                   36407
Resolved              25751
Closed                24985
Awaiting User Info    14642
Awaiting Vendor         707
Awaiting Problem        461
Awaiting Evidence        38
-100                      5
Name: count, dtype: int64

incident_state
New                   16397
Active                 6566
Resolved               1816
Awaiting User Info      133
Awaiting Problem          6
Name: count, dtype: int64

## list of columns not used in training set
* 

## create a data frame with the following columns for training
#List of columns to keep for Training set:
* opened_by: identifier of the user who reported the incident
* opened_at: incident user opening date and time
* contact_type: categorical attribute that shows by what means the incident was reported
* location: identifier of the location of the place affected
* category: first-level description of the affected service
* subcategory: second-level description of the affected service (related to the first level description, i.e., to category);
* u_symptom: description of the user perception about service availability
* impact: description of the impact caused by the incident (values: 1â€“High; 2â€“Medium; 3â€“Low);
* urgency: description of the urgency informed by the user for the incident resolution (values: 1â€“High; 2â€“Medium; 3â€“Low);
* priority: calculated by the system based on 'impact' and 'urgency';
* assignment_group: identifier of the support group in charge of the incident;
* assigned_to: identifier of the user in charge of the incident;
* knowledge: boolean attribute that shows whether a knowledge base document was used to resolve the incident;
* u_priority_confirmation: boolean attribute that shows whether the priority field has been double-checked;
* notify: categorical attribute that shows whether notifications were generated for the incident;

### Too many missing values?
* rfc: (98 entries)(request for change) identifier of the change request associated with the incident;
* cmdb_ci: (only has 51 entries) (confirmation item) identifier used to report the affected item (not mandatory)
* vendor:(probably drop only 15 entries) identifier of the vendor in charge of the incident;
* caused_by:(drop only 2 entires) identifier of the RFC responsible by the incident
* problem_id:(241 entries) identifier of the problem associated with the incident;
*  sys_created_by: identifier of the user who registered the incident
* sys_created_at: incident system creation date and time


list of potential:exclude these columns
* sys_updated_by:(? is this generated day 1?) identifier of the user who updated the incident and generated the current log record
* sys_updated_at:(?day 1 info?) incident system update date and time


In [162]:
train_df = df_new_active[['opened_by', 'contact_type','location',
                                'category', 'subcategory','u_symptom','impact','urgency', 'priority',
                                'assignment_group','assigned_to','knowledge', 'u_priority_confirmation', 'notify',
                                    ]]
train_df.head()

,opened_by,contact_type,location,category,subcategory,u_symptom,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify
0,Opened by 8,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,Group 56,NaN,True,False,Do Not Notify
1,Opened by 397,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 89,True,False,Do Not Notify
2,Opened by 8,Phone,Location 204,Category 20,Subcategory 125,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,Group 70,NaN,True,False,Do Not Notify
3,Opened by 180,Phone,Location 204,Category 9,Subcategory 97,Symptom 450,2 - Medium,2 - Medium,3 - Moderate,Group 25,Resolver 125,True,False,Do Not Notify
4,Opened by 180,Phone,Location 93,Category 53,Subcategory 168,Symptom 232,2 - Medium,2 - Medium,3 - Moderate,Group 70,NaN,True,False,Do Not Notify


**Who are we presenting too:** ServiceNow IT department

#### Target
* create target column: numbers of days ticket is open???
* what could this tell us, or how could we explain this as usefull: if we can succesfully predict whether an incident will be resolved in sufficent time given info porovided about the incident. then we could advise...???
* who wants this

#### TO DO List
* **complete** figure out how to filter to find all duplicate incedents for same value in 'number' and only retain 1 incedence set to 'incident_status == 'new'
* **complete**create repository and seperate branches  link: https://github.com/Sug-ar-N-Spice/Incident_Events.git
* **complete**turn all ? into NANs
* **complete**find documentation/info of columns   link: https://archive.ics.uci.edu/dataset/498/incident+management+process+enriched+event+log
* **semi-complete**create target column: complete for tickets open over 2 days
* **complete** decide if there are any columns we can get drop ..ex.'number'
* **complete** also decide what columns would cuase dataleaks to give the model all the answers renderring our model useless..ex. time/date, days open, other columns?
* decide if the target column is relavant enough for conclusion
* Parameter loops
* do we need fucntions? because this is our only dataset, we don't have another one to test?
* DO we need a standard scaler(need it)
* DO we need PCA to reduce the columns?
* Do we need resampler(need it)
* clustering? dont we need it for certain models? #original task is regresion and clustering should we do this to understand data

**curenlty working on:**
* **complete** list all columns with NaN values
* find the NANS and figure out how to fill
* replace strings with numbers
* list out the models that we will attempt to use find code examples/syntax ready
  

### Converting strings into numbers:
* seperate dates into 2 to 3 columns: days of week, days of year
* Try to look up code references, but randomforestclassfier might be able to overcome and just keep code numbers



### columns filled with grab_1rst_num
possibly check for pass and fails, This is there human process for deciding how imporant an incident is
* urgency
* priority
* impact


**use this code when ready convert all objects to numbers**
train_df = train_df.apply(lambda x: x.astype(int) if x.dtype == "object" else x)

### Filling missing NANs:
*sys_created_at and sys_created_by have the same missing values(took out of dataset)
####Function(grab_number_last)
* opened_by: assign to -1 (714)
* location: fill with -1   (filling 51)
* category:-1 (filling 39)
* subcategory: -1
* u_symptom: -1
* assignment_group: -1


* contact_type: fill with 'unknown'
* assigned_to
* knowledge
* u_priority_confirmation
* notify


original columns with NAN
['opened_by',
 'location',
 'category',
 'subcategory',
 'u_symptom',
 'assignment_group',
 'assigned_to']


In [163]:
def grab_number(value):
    #print (value) 
    if (pd.isna(value)).any():
        ### isna is nan values
        return True
    return False

# df['caller_id'].str.split(" ").apply(lambda x: x[0]).value_counts()
#sum(df.apply(grab_number)) #.astype(str)
df_nan.apply(grab_number)

number                     False
incident_state             False
active                     False
reassignment_count         False
reopen_count               False
sys_mod_count              False
made_sla                   False
caller_id                   True
opened_by                   True
opened_at                  False
sys_created_by              True
sys_created_at              True
sys_updated_by             False
sys_updated_at             False
contact_type               False
location                    True
category                    True
subcategory                 True
u_symptom                   True
cmdb_ci                     True
impact                     False
urgency                    False
priority                   False
assignment_group            True
assigned_to                 True
knowledge                  False
u_priority_confirmation    False
notify                     False
problem_id                  True
rfc                         True
vendor    

In [164]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24918 entries, 0 to 24917
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   opened_by                24204 non-null  object
 1   contact_type             24918 non-null  object
 2   location                 24866 non-null  object
 3   category                 24879 non-null  object
 4   subcategory              24849 non-null  object
 5   u_symptom                19182 non-null  object
 6   impact                   24918 non-null  object
 7   urgency                  24918 non-null  object
 8   priority                 24918 non-null  object
 9   assignment_group         24004 non-null  object
 10  assigned_to              17772 non-null  object
 11  knowledge                24918 non-null  bool  
 12  u_priority_confirmation  24918 non-null  bool  
 13  notify                   24918 non-null  object
dtypes: bool(2), object(12)
memory usage: 2

In [165]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24918 entries, 0 to 24917
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   opened_by                24204 non-null  object
 1   contact_type             24918 non-null  object
 2   location                 24866 non-null  object
 3   category                 24879 non-null  object
 4   subcategory              24849 non-null  object
 5   u_symptom                19182 non-null  object
 6   impact                   24918 non-null  object
 7   urgency                  24918 non-null  object
 8   priority                 24918 non-null  object
 9   assignment_group         24004 non-null  object
 10  assigned_to              17772 non-null  object
 11  knowledge                24918 non-null  bool  
 12  u_priority_confirmation  24918 non-null  bool  
 13  notify                   24918 non-null  object
dtypes: bool(2), object(12)
memory usage: 2

In [166]:
def grab_number(value):
    #print (value) 
    if (pd.isna(value)).any():
        ### isna is nan values
        return True
    return False

# df['caller_id'].str.split(" ").apply(lambda x: x[0]).value_counts()
#sum(df.apply(grab_number)) #.astype(str)
sum(train_df.apply(grab_number))

7

In [167]:
def grab_number(value):
    #print (value) 
    if (value == "?").any():
        ### this code is saying go through each and every column and for each row try to find if that row is equal to Caller 2403
        return True
    return False

# df['caller_id'].str.split(" ").apply(lambda x: x[0]).value_counts()
#sum(df.apply(grab_number)) #.astype(str)
sum(train_df.apply(grab_number))

0

In [168]:
# def grab_number_last(value):
#     """parameter: value enter in a column from the data frame where you want to capture the last string and convert to number
#     by converting all values to string, splitting them, and then only capturing the last string
     
#     Retruns: last string and if NaN returns -1
      
#     call: convert the column using df.loc[:,"column_name"] = df["column_name"].apply(grab_number_3rd).astype(int)
#     This avoids warnings and converts to an integer"""
    
    
#     if pd.isna(value):
#         return -1  # converts NaN values to -1
#     return str(value).split(" ")[-1]  # splits the string and then captures the last value 


# # apply the function to the desired column and convert the result to integers
# #use the loc function to specify the column being converted to avoid warnings of working on slice of dataframe
# train_df.loc[:, 'opened_by'] = train_df['opened_by'].apply(grab_number_last).astype(int)
# train_df.loc[:, 'location'] = train_df['location'].apply(grab_number_last).astype(int)
# train_df.loc[:, 'category'] = train_df['category'].apply(grab_number_last).astype(int)
# train_df.loc[:, 'subcategory'] = train_df['subcategory'].apply(grab_number_last).astype(int)
# train_df.loc[:, 'u_symptom'] = train_df['u_symptom'].apply(grab_number_last).astype(int)
# train_df.loc[:, 'assignment_group'] = train_df['assignment_group'].apply(grab_number_last).astype(int)
# train_df.loc[:, 'assigned_to'] = train_df['assigned_to'].apply(grab_number_last).astype(int)



In [169]:
def grab_number_last(value):
    """parameter: value enter in a column from the data frame where you want to capture the last string and convert to number
    by converting all values to string, splitting them, and then only capturing the last string
     
    Retruns: last string and if NaN returns -1
      
    call: convert the column using df.loc[:,"column_name"] = df["column_name"].apply(grab_number_3rd).astype(int)
    This avoids warnings and converts to an integer"""
    
    
    if pd.isna(value):
        return -1  # converts NaN values to -1
    return str(value).split(" ")[-1]  # splits the string and then captures the last value 


# apply the function to the desired column and convert the result to integers
#use the loc function to specify the column being converted to avoid warnings of working on slice of dataframe
train_df.loc[:, 'opened_by'] = train_df['opened_by'].apply(grab_number_last).astype(int)
train_df.loc[:, 'location'] = train_df['location'].apply(grab_number_last).astype(int)
train_df.loc[:, 'category'] = train_df['category'].apply(grab_number_last).astype(int)
train_df.loc[:, 'subcategory'] = train_df['subcategory'].apply(grab_number_last).astype(int)
train_df.loc[:, 'u_symptom'] = train_df['u_symptom'].apply(grab_number_last).astype(int)
train_df.loc[:, 'assignment_group'] = train_df['assignment_group'].apply(grab_number_last).astype(int)
train_df.loc[:, 'assigned_to'] = train_df['assigned_to'].apply(grab_number_last).astype(int)


In [170]:
def grab_number(value):
    #print (value) 
    if (pd.isna(value)).any():
        ### isna is nan values
        return True
    return False

# df['caller_id'].str.split(" ").apply(lambda x: x[0]).value_counts()
#sum(df.apply(grab_number)) #.astype(str)
train_df.apply(grab_number)

opened_by                  False
contact_type               False
location                   False
category                   False
subcategory                False
u_symptom                  False
impact                     False
urgency                    False
priority                   False
assignment_group           False
assigned_to                False
knowledge                  False
u_priority_confirmation    False
notify                     False
dtype: bool

In [171]:
train_df['opened_by'].dtype

dtype('O')

In [178]:
def grab_1rst_num(value):
    if (value):
        return str(value).split(" ")[0]

train_df['urgency'] = train_df['urgency'].apply(grab_1rst_num).astype(int)
train_df['impact'] = train_df['impact'].apply(grab_1rst_num).astype(int)
train_df['priority'] = train_df['priority'].apply(grab_1rst_num).astype(int)

# :,   
## this process is converting intehers per row and not the whole series. if want seried to be integer then covert 
train_df.dtypes



/var/folders/ys/j34f3pms18l5jc7tq2nl1qn00000gp/T/ipykernel_13857/2886423002.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['urgency'] = train_df['urgency'].apply(grab_1rst_num).astype(int)
/var/folders/ys/j34f3pms18l5jc7tq2nl1qn00000gp/T/ipykernel_13857/2886423002.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['impact'] = train_df['impact'].apply(grab_1rst_num).astype(int)
/var/folders/ys/j34f3pms18l5jc7tq2nl1qn00000gp/T/ipykernel_13857/2886423002.py:7: SettingWithCopyWarning: 
A

opened_by                  object
contact_type               object
location                   object
category                   object
subcategory                object
u_symptom                  object
impact                      int64
urgency                     int64
priority                    int64
assignment_group           object
assigned_to                object
knowledge                   int64
u_priority_confirmation     int64
notify                     object
dtype: object

In [177]:
def grab_1rst_num(value):
    if (value):
        return str(value).split(" ")[0]

train_df.loc[:,'urgency'] = train_df['urgency'].apply(grab_1rst_num).astype(int)
train_df.loc[:,'impact'] = train_df['impact'].apply(grab_1rst_num).astype(int)
train_df.loc[:,'priority'] = train_df['priority'].apply(grab_1rst_num).astype(int)

# :,   
## this process is converting intehers per row and not the whole series. if want seried to be integer then covert 
train_df.dtypes



opened_by                  object
contact_type               object
location                   object
category                   object
subcategory                object
u_symptom                  object
impact                     object
urgency                     int64
priority                   object
assignment_group           object
assigned_to                object
knowledge                   int64
u_priority_confirmation     int64
notify                     object
dtype: object

In [174]:
##LabelEncoder is a function in sklearn to change boolean column to zero and 1 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit_transform(train_df['knowledge'])
# train_df['knowledge']

array([1, 1, 1, ..., 0, 0, 0])

In [175]:
def onehot_bool(value):
    if value == True:
        return int(0)
    return int(1)


train_df['knowledge'] = train_df['knowledge'].apply(onehot_bool)
train_df['u_priority_confirmation'] = train_df['u_priority_confirmation'].apply(onehot_bool)

# #train_df.loc[:, 'knowledge'] == train_df['knowledge'].apply(onehot_bool).astype(int)

/var/folders/ys/j34f3pms18l5jc7tq2nl1qn00000gp/T/ipykernel_13857/1418183835.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['knowledge'] = train_df['knowledge'].apply(onehot_bool)
/var/folders/ys/j34f3pms18l5jc7tq2nl1qn00000gp/T/ipykernel_13857/1418183835.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['u_priority_confirmation'] = train_df['u_priority_confirmation'].apply(onehot_bool)


In [176]:
train_dum_df['u_priority_confirmation'].value_counts()

NameError: name 'train_dum_df' is not defined

In [ ]:
train_df.head()

,opened_by,contact_type,location,category,subcategory,u_symptom,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify
0,8,Phone,143,55,170,72,2,2,3,56,-1,0,False,Do Not Notify
1,397,Phone,165,40,215,471,2,2,3,70,89,0,False,Do Not Notify
2,8,Phone,204,20,125,471,2,2,3,70,-1,0,False,Do Not Notify
3,180,Phone,204,9,97,450,2,2,3,25,125,0,False,Do Not Notify
4,180,Phone,93,53,168,232,2,2,3,70,-1,0,False,Do Not Notify


In [ ]:
train_dum_df = train_df.copy()

dummies = pd.get_dummies(train_dum_df[['contact_type','notify']], drop_first=True, dtype=int)

train_dum_df = train_dum_df.drop(['contact_type','notify'], axis = 1)

train_dum_df = pd.concat([train_dum_df, dummies], axis=1)

train_dum_df.head()

,opened_by,location,category,subcategory,u_symptom,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,contact_type_Email,contact_type_IVR,contact_type_Phone,contact_type_Self service,notify_Send Email
0,8,143,55,170,72,2,2,3,56,-1,1,1,0,0,1,0,0
1,397,165,40,215,471,2,2,3,70,89,1,1,0,0,1,0,0
2,8,204,20,125,471,2,2,3,70,-1,1,1,0,0,1,0,0
3,180,204,9,97,450,2,2,3,25,125,1,1,0,0,1,0,0
4,180,93,53,168,232,2,2,3,70,-1,1,1,0,0,1,0,0


In [ ]:
train_dum_df = train_dum_df.apply(lambda x: x.astype(int) if x.dtype == "object" else x)

In [ ]:
train_dum_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24918 entries, 0 to 24917
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   opened_by                  24918 non-null  int32
 1   location                   24918 non-null  int32
 2   category                   24918 non-null  int32
 3   subcategory                24918 non-null  int32
 4   u_symptom                  24918 non-null  int32
 5   impact                     24918 non-null  int32
 6   urgency                    24918 non-null  int32
 7   priority                   24918 non-null  int32
 8   assignment_group           24918 non-null  int32
 9   assigned_to                24918 non-null  int32
 10  knowledge                  24918 non-null  int64
 11  u_priority_confirmation    24918 non-null  int64
 12  contact_type_Email         24918 non-null  int32
 13  contact_type_IVR           24918 non-null  int32
 14  contact_type_Phone    

In [ ]:
#check to make sure all NaNs are gone
train_df['opened_by'].isnull().sum()


0

In [ ]:
# Find the percentage of null values in each column
train_df.isna().sum()/len(train_df)

opened_by                  0.000000
contact_type               0.000000
location                   0.000000
category                   0.000000
subcategory                0.000000
u_symptom                  0.000000
impact                     0.000000
urgency                    0.000000
priority                   0.000000
assignment_group           0.000000
assigned_to                0.286781
knowledge                  0.000000
u_priority_confirmation    0.000000
notify                     0.000000
dtype: float64

In [ ]:
columns_with_nan = train_df.columns[train_df.isna().any()].tolist()
columns_with_nan

['assigned_to']

In [ ]:
#create empty dictionary to store stats tables for all nan columns
nan_column_stats = {}

#loop through each column in the list of columns with nan values
for col in columns_with_nan:
    #using the loc function to list the rows with Nan values in the particular column and calculate descpritive stats for all columns
    nan_column_stats[col] = df_nan.loc[df_nan[col].isna()].describe(include='all')
    pd.set_option('display.max_columns', None)

In [ ]:
#display the stats for a specific column
nan_column_stats['opened_by']

nan_column_stats['assigned_to']

nan_column_stats['assignment_group']

nan_column_stats['u_symptom']

nan_column_stats['subcategory']

nan_column_stats['category']

nan_column_stats['location']

In [ ]:
# Awaiting User Info      133
# Awaiting Problem          6
df_awaiting = df_new_active.loc[
    (df_new_active['incident_state'] == 'Awaiting User Info') | 
    (df_new_active['incident_state'] == 'Awaiting Problem')]
df_awaiting
df_awaiting.value_counts('target_colum')

target_colum
1    102
0     37
Name: count, dtype: int64

In [ ]:
df_made_sla = df_new_active.loc[df_new_active['made_sla'] == False]
df_made_sla

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,difference,target_colum
1630,INC0002588,Active,True,0,0,0,False,Caller 3571,Opened by 325,2016-03-04 15:00:00,...,NaN,NaN,NaN,NaN,code 1,Resolved by 143,2016-03-11 10:38:00,24/3/2016 18:50,6.0,1


In [ ]:
#shows a measurement of how many times they under performed in resolving the incident in the time specified in their contract with the client
df_nan.value_counts('made_sla')
#132497 / (9215+132497) * 100
#9215 / (9215+132497) * 100



made_sla
True     132497
False      9215
Name: count, dtype: int64

In [ ]:
df_u_confirmation = df_new_active.loc[df_new_active['u_priority_confirmation'] == True]
df_u_confirmation.head()

In [ ]:
df_priotity = df_new_active.loc[df_new_active['priority'] == '1 - Critical']
df_priotity



In [ ]:
df_new_active.info()

In [ ]:
df_closed_code = df_new_active.loc[df_new_active['closed_code'] == 'code 7']
df_closed_code

In [ ]:
df_new_active['closed_code'].isna().count()

In [ ]:
df_new_active[['opened_at', 'sys_updated_at']]

In [ ]:
df_new_active.value_counts('closed_code')

In [ ]:
df_new_active.loc[df_new_active['reopen_count'] == 1]

In [ ]:
df_new_active.value_counts('reopen_count')


reopen_count
0    24915
1        3
Name: count, dtype: int64

In [ ]:
df_inc = df.loc[df['number'] == 'INC0011551']
df_inc

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
50379,INC0011551,Resolved,True,0,0,0,True,Caller 5589,Opened by 17,27/3/2016 08:30,?,?,Updated by 908,27/3/2016 08:30,Phone,Location 204,Category 32,Subcategory 174,Symptom 491,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 13,False,False,Do Not Notify,?,?,?,?,code 7,Resolved by 11,?,1/4/2016 08:59
50380,INC0011551,Closed,False,0,0,1,True,Caller 5589,Opened by 17,27/3/2016 08:30,?,?,Updated by 908,1/4/2016 08:59,Phone,Location 204,Category 32,Subcategory 174,Symptom 491,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 13,False,False,Do Not Notify,?,?,?,?,code 7,Resolved by 11,?,1/4/2016 08:59


In [ ]:
df_inc0000093 = df_nan.loc[df_nan['number'] == 'INC0000093']
df_inc0000093
""" 'sys_updated_at' , sys_mod_count, & 'active' are the different columns"""

" 'sys_updated_at' , sys_mod_count& 'active' are the different columns"

In [ ]:
df_resolved = df_new_active.loc[df_new_active['incident_state'] == 'Resolved']
df_resolved.value_counts('target_colum')

target_colum
0    1815
1       1
Name: count, dtype: int64

In [ ]:
df_inc0027738 = df_nan[df_nan['number'] == 'INC0027738']
df_inc0027738

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,difference,target_colum
114280,INC0027738,Active,True,0,0,0,True,Caller 3368,Opened by 131,2016-05-10 08:48:00,NaN,NaN,Updated by 908,10/5/2016 08:48,Phone,Location 93,Category 42,Subcategory 223,Symptom 534,NaN,3 - Low,3 - Low,4 - Low,Group 70,Resolver 73,False,False,Do Not Notify,NaN,NaN,NaN,NaN,code 7,Resolved by 66,2016-05-10 09:37:00,15/5/2016 10:07,0.0,0
114281,INC0027738,Resolved,True,0,0,1,True,Caller 3368,Opened by 131,2016-05-10 08:48:00,NaN,NaN,Updated by 265,10/5/2016 09:37,Phone,Location 93,Category 42,Subcategory 223,Symptom 534,NaN,3 - Low,3 - Low,4 - Low,Group 70,Resolver 73,False,True,Do Not Notify,NaN,NaN,NaN,NaN,code 7,Resolved by 66,2016-05-10 09:37:00,15/5/2016 10:07,0.0,0
114282,INC0027738,Closed,False,0,0,2,True,Caller 3368,Opened by 131,2016-05-10 08:48:00,NaN,NaN,Updated by 908,15/5/2016 10:07,Phone,Location 93,Category 42,Subcategory 223,Symptom 534,NaN,3 - Low,3 - Low,4 - Low,Group 70,Resolver 73,False,True,Do Not Notify,NaN,NaN,NaN,NaN,code 7,Resolved by 66,2016-05-10 09:37:00,15/5/2016 10:07,0.0,0


In [ ]:
df_inc0019396 = df[df['number'] == 'INC0019396']
display(df_inc0019396.head())
display(df_inc0019396.tail())

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
81418,INC0019396,New,True,0,0,0,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 908,14/4/2016 20:42,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 66,?,False,False,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81419,INC0019396,New,True,0,0,1,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 278,14/4/2016 21:08,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 66,?,False,False,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81420,INC0019396,Resolved,True,0,0,6,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 713,15/4/2016 09:20,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 66,Resolver 175,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81421,INC0019396,Active,True,0,1,7,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 278,15/4/2016 15:21,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 66,Resolver 175,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81422,INC0019396,Active,True,0,1,9,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 278,15/4/2016 15:21,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 66,Resolver 175,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07


,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
81471,INC0019396,Resolved,True,10,7,63,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 429,12/5/2016 14:46,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 115,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81472,INC0019396,Active,True,10,8,64,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 278,12/5/2016 16:30,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 115,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81473,INC0019396,Active,True,10,8,66,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 278,12/5/2016 16:30,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 115,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81474,INC0019396,Resolved,True,10,8,67,True,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 111,12/5/2016 23:51,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 26,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07
81475,INC0019396,Closed,False,10,8,68,False,Caller 1580,Opened by 305,14/4/2016 20:42,?,?,Updated by 908,18/5/2016 00:07,Phone,Location 143,Category 53,Subcategory 114,?,?,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 26,False,True,Do Not Notify,?,?,?,?,code 6,Resolved by 158,12/5/2016 23:51,18/5/2016 00:07


In [ ]:
display(df.head())
display(df.tail())

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,target_colum,difference
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,29/2/2016 01:23,Updated by 21,29/2/2016 01:23,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,NaN,True,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,5/3/2016 12:00,0,0.0
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,29/2/2016 01:23,Updated by 642,29/2/2016 08:53,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,NaN,True,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,5/3/2016 12:00,0,0.0
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,29/2/2016 01:23,Updated by 804,29/2/2016 11:29,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,NaN,True,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,5/3/2016 12:00,0,0.0
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,Created by 6,29/2/2016 01:23,Updated by 908,5/3/2016 12:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,NaN,True,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 149,2016-02-29 11:29:00,5/3/2016 12:00,0,0.0
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,2016-02-29 04:40:00,Created by 171,29/2/2016 04:57,Updated by 746,29/2/2016 04:57,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 89,True,False,Do Not Notify,NaN,NaN,NaN,NaN,code 5,Resolved by 81,2016-03-01 09:52:00,6/3/2016 10:00,0,1.0


,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,target_colum,difference
141707,INC0120835,Closed,False,1,0,4,True,Caller 116,Opened by 12,2017-02-16 09:09:00,NaN,NaN,Updated by 27,16/2/2017 09:53,Email,Location 204,Category 42,Subcategory 223,Symptom 494,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 31,Resolver 10,False,True,Do Not Notify,NaN,NaN,NaN,NaN,code 9,Resolved by 9,2017-02-16 09:53:00,16/2/2017 09:53,0,0.0
141708,INC0121064,Active,True,0,0,0,True,Caller 116,Opened by 12,2017-02-16 14:17:00,NaN,NaN,Updated by 908,16/2/2017 14:17,Email,Location 204,Category 42,Subcategory 223,Symptom 494,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 70,Resolver 10,False,False,Do Not Notify,NaN,NaN,NaN,NaN,code 6,Resolved by 9,2017-02-16 16:38:00,16/2/2017 16:38,0,0.0
141709,INC0121064,Active,True,1,0,1,True,Caller 116,Opened by 12,2017-02-16 14:17:00,NaN,NaN,Updated by 60,16/2/2017 15:20,Email,Location 204,Category 42,Subcategory 223,Symptom 494,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 31,NaN,False,False,Do Not Notify,NaN,NaN,NaN,NaN,code 6,Resolved by 9,2017-02-16 16:38:00,16/2/2017 16:38,0,0.0
141710,INC0121064,Resolved,True,1,0,2,True,Caller 116,Opened by 12,2017-02-16 14:17:00,NaN,NaN,Updated by 27,16/2/2017 16:38,Email,Location 204,Category 42,Subcategory 223,Symptom 494,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 31,Resolver 10,False,True,Do Not Notify,NaN,NaN,NaN,NaN,code 6,Resolved by 9,2017-02-16 16:38:00,16/2/2017 16:38,0,0.0
141711,INC0121064,Closed,False,1,0,3,True,Caller 116,Opened by 12,2017-02-16 14:17:00,NaN,NaN,Updated by 27,16/2/2017 16:38,Email,Location 204,Category 42,Subcategory 223,Symptom 494,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 31,Resolver 10,False,True,Do Not Notify,NaN,NaN,NaN,NaN,code 6,Resolved by 9,2017-02-16 16:38:00,16/2/2017 16:38,0,0.0


In [ ]:
df.value_counts('active')

active
True     116726
False     24986
Name: count, dtype: int64

In [ ]:
df.value_counts('incident_state')

incident_state
Active                38716
New                   36407
Resolved              25751
Closed                24985
Awaiting User Info    14642
Awaiting Vendor         707
Awaiting Problem        461
Awaiting Evidence        38
-100                      5
Name: count, dtype: int64

In [ ]:
df.value_counts('opened_by')

opened_by
Opened by  17     41466
Opened by  24      8063
Opened by  131     7269
Opened by  108     6121
Opened by  55      5655
                  ...  
Opened by  447        3
Opened by  80         3
Opened by  440        3
Opened by  486        3
Opened by  144        3
Name: count, Length: 207, dtype: int64

In [ ]:
df.columns


Index(['number', 'incident_state', 'active', 'reassignment_count',
       'reopen_count', 'sys_mod_count', 'made_sla', 'caller_id', 'opened_by',
       'opened_at', 'sys_created_by', 'sys_created_at', 'sys_updated_by',
       'sys_updated_at', 'contact_type', 'location', 'category', 'subcategory',
       'u_symptom', 'cmdb_ci', 'impact', 'urgency', 'priority',
       'assignment_group', 'assigned_to', 'knowledge',
       'u_priority_confirmation', 'notify', 'problem_id', 'rfc', 'vendor',
       'caused_by', 'closed_code', 'resolved_by', 'resolved_at', 'closed_at'],
      dtype='object')

In [ ]:
df.value_counts('impact')

impact
2 - Medium    134335
3 - Low         3886
1 - High        3491
Name: count, dtype: int64

In [ ]:
df.value_counts('incident_state')

incident_state
Active                38716
New                   36407
Resolved              25751
Closed                24985
Awaiting User Info    14642
Awaiting Vendor         707
Awaiting Problem        461
Awaiting Evidence        38
-100                      5
Name: count, dtype: int64

In [ ]:
df.value_counts('active')

active
True     116726
False     24986
Name: count, dtype: int64

In [ ]:
df.value_counts('reopen_count')

reopen_count
0    139398
1      1918
2       190
3       107
4        48
5        24
6        18
7         5
8         4
Name: count, dtype: int64

In [ ]:
df.value_counts('incident_state', 'active')

incident_state
Active                0.273202
New                   0.256908
Resolved              0.181714
Closed                0.176308
Awaiting User Info    0.103322
Awaiting Vendor       0.004989
Awaiting Problem      0.003253
Awaiting Evidence     0.000268
-100                  0.000035
Name: proportion, dtype: float64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141712 entries, 0 to 141711
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   number                   141712 non-null  object
 1   incident_state           141712 non-null  object
 2   active                   141712 non-null  bool  
 3   reassignment_count       141712 non-null  int64 
 4   reopen_count             141712 non-null  int64 
 5   sys_mod_count            141712 non-null  int64 
 6   made_sla                 141712 non-null  bool  
 7   caller_id                141712 non-null  object
 8   opened_by                141712 non-null  object
 9   opened_at                141712 non-null  object
 10  sys_created_by           141712 non-null  object
 11  sys_created_at           141712 non-null  object
 12  sys_updated_by           141712 non-null  object
 13  sys_updated_at           141712 non-null  object
 14  contact_type        

In [ ]:
display(df.value_counts('problem_id'))
display(df.value_counts('rfc'))
display(df.value_counts('vendor'))
display(df.value_counts('caused_by'))

problem_id
?                  139417
Problem ID  14        184
Problem ID  2         147
Problem ID  52         82
Problem ID  239        48
                    ...  
Problem ID  181         1
Problem ID  88          1
Problem ID  11          1
Problem ID  195         1
Problem ID  33          1
Name: count, Length: 253, dtype: int64

rfc
?             140721
CHG0000132        20
CHG0001230        20
CHG0000047        18
CHG0001656        17
               ...  
CHG0000207         2
CHG0001829         1
CHG0000530         1
CHG0001437         1
CHG0001720         1
Name: count, Length: 182, dtype: int64

vendor
?           141468
code 8s        167
Vendor 1        69
Vendor 3         6
Vendor 2         2
Name: count, dtype: int64

caused_by
?             141689
CHG0000097        11
CHG0000132         7
CHG0001327         5
Name: count, dtype: int64

In [ ]:
display(df.value_counts('opened_at'))
display(df.value_counts('resolved_at'))

In [ ]:
df['resolved_at'].isnull().value_counts()

resolved_at
False    138571
True       3141
Name: count, dtype: int64

In [ ]:
df['resolved_at'].value_counts()

In [ ]:
df_nan.head()

In [ ]:
check_dates =df_nan[['opened_at','resolved_at']]
check_dates.head(50)

In [ ]:
df_nan.info()

In [ ]:
#This drops all columns with Nan values don't know what we would use this for
df_drop = df_nan.dropna(axis = 1).copy()
df_drop.info()

In [ ]:
#need to convert all strings to numbers before this code will work
df_nan.corr()['target'].sort_values()